In [1]:
import numpy as np
import pandas as pd

We create our first function, which will unnest any json text into plain text

In [2]:
def unnest(path, dicc):
    """
    This function allows us to read json text as plain.
    
    Parameters
    ----------
    path : str
        path where the json file is located.
    dicc : dict
        dictionary with key-value pairs.

    returns
    -------
    DataFrame
        result of converting said dictionary to a DataFrame.

    Examples
    --------
    >>> unnest('./folder/file.csv', dictionary)
    {
    "names": ["Michael", "Jim", "Dwight"], 
    "Position": ["Manager", "Salesman", "Salesman"],
    "Age": [42, 29, 27]
    }
    """

    df = []
    with open(path, encoding = 'UTF-8-SIG') as f:
        for line in f:
            df.append(eval(line))
    
    for i in df:
        for clave, valor in i.items():
            dicc[clave].append(valor)
    dicc = pd.DataFrame(dicc)
    return dicc

In [3]:
# We make our dictionary with key names same as the column names and an empty list.
diccionarioReviews = {'user_id' : [], 'user_url': [], 'reviews': []}

reviews = unnest('./DataSets/australian_user_reviews.json', diccionarioReviews)

In [4]:
reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [5]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


We drop any duplicate value our dataset may have

In [6]:
reviews[reviews['user_id'].duplicated(keep = False)].sort_values(by = 'user_id')

,user_id,user_url,reviews
12888,05041129,http://steamcommunity.com/id/05041129,"[{'funny': '', 'posted': 'Posted May 18, 2015...."
5250,05041129,http://steamcommunity.com/id/05041129,"[{'funny': '', 'posted': 'Posted May 18, 2015...."
3133,111222333444555666888,http://steamcommunity.com/id/11122233344455566...,"[{'funny': '', 'posted': 'Posted December 22, ..."
3134,111222333444555666888,http://steamcommunity.com/id/11122233344455566...,"[{'funny': '', 'posted': 'Posted December 22, ..."
4139,29123,http://steamcommunity.com/id/29123,"[{'funny': '', 'posted': 'Posted March 26.', '..."
...,...,...,...
2721,xXAussieRockXx,http://steamcommunity.com/id/xXAussieRockXx,"[{'funny': '', 'posted': 'Posted July 17, 2015..."
2680,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
17916,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
5855,zeroblade,http://steamcommunity.com/id/zeroblade,"[{'funny': '', 'posted': 'Posted November 30, ..."


In [7]:
reviews.drop_duplicates(subset = 'user_id', keep = False, inplace = True)

In [8]:
reviews.head().sort_values(by = 'user_id')

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


We repeat this process with our items DataSet.

In [9]:
items = {'user_id' : [], 'items_count': [], 'steam_id': [], 'user_url': [], 'items': []}
items = unnest('./DataSets/australian_users_items.json', items)


In [10]:
items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [11]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


We check for any duplicate value.

In [12]:
items[items['user_id'].duplicated(keep = False)].sort_values(by = 'user_id')

,user_id,items_count,steam_id,user_url,items
11000,05041129,35,76561198167088451,http://steamcommunity.com/id/05041129,"[{'item_id': '4000', 'item_name': 'Garry's Mod..."
29193,05041129,35,76561198167088451,http://steamcommunity.com/id/05041129,"[{'item_id': '4000', 'item_name': 'Garry's Mod..."
37062,10outof10matee,56,76561198050688208,http://steamcommunity.com/id/10outof10matee,"[{'item_id': '220', 'item_name': 'Half-Life 2'..."
37061,10outof10matee,56,76561198050688208,http://steamcommunity.com/id/10outof10matee,"[{'item_id': '220', 'item_name': 'Half-Life 2'..."
6167,111222333444555666888,52,76561198082607692,http://steamcommunity.com/id/11122233344455566...,"[{'item_id': '240', 'item_name': 'Counter-Stri..."
...,...,...,...,...,...
4625,youseeitnowgetout,5,76561198087136132,http://steamcommunity.com/id/youseeitnowgetout,"[{'item_id': '230410', 'item_name': 'Warframe'..."
3473,zandado,107,76561198057890701,http://steamcommunity.com/id/zandado,"[{'item_id': '20', 'item_name': 'Team Fortress..."
34176,zandado,107,76561198057890701,http://steamcommunity.com/id/zandado,"[{'item_id': '20', 'item_name': 'Team Fortress..."
12417,zeroblade,306,76561197970272666,http://steamcommunity.com/id/zeroblade,"[{'item_id': '18110', 'item_name': 'Shattered ..."


We drop said values

In [13]:
items.drop_duplicates(subset = 'user_id', keep = 'first', inplace = True)

We make our Steam DataSet

In [14]:
df = []

with open('./DataSets/output_steam_games.json', encoding = 'UTF-8-SIG') as f:
    for line in f:
        if '"id": NaN'in line:
            continue
        line = line.replace("true", "True")
        line = line.replace("false", "False")
        line = line.replace("NaN", "None")
        df.append(eval(line))

steam = pd.DataFrame(df)

In [15]:
steam[steam['title'].duplicated(keep = False)].sort_values(by = 'title')

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
30178,SEGA,[Action],Aliens: Colonial Marines - Reconnaissance Pack,Aliens: Colonial Marines - Reconnaissance Pack,http://store.steampowered.com/app/219441/Alien...,2013-05-07,[Action],http://steamcommunity.com/app/219441/reviews/?...,"[Single-player, Multi-player, Co-op, Downloada...",29.99,False,219441,Gearbox Software
30177,SEGA,[Action],Aliens: Colonial Marines - Reconnaissance Pack,Aliens: Colonial Marines - Reconnaissance Pack,http://store.steampowered.com/app/224850/Alien...,2013-05-07,[Action],http://steamcommunity.com/app/224850/reviews/?...,"[Single-player, Multi-player, Co-op, Downloada...",None,False,224850,Gearbox Software
31401,Viva Media,[Adventure],Alter Ego,Alter Ego,http://store.steampowered.com/app/63110/Alter_...,2010-08-03,"[Adventure, Point & Click, Mystery, Detective]",http://steamcommunity.com/app/63110/reviews/?b...,[Single-player],9.99,False,63110,bitComposer Games
10912,Choose Multiple LLC,"[Adventure, Casual, Indie, RPG, Simulation]",Alter Ego,Alter Ego,http://store.steampowered.com/app/664780/Alter...,1986-05-01,"[RPG, Casual, Indie, Adventure, Simulation, Te...",http://steamcommunity.com/app/664780/reviews/?...,"[Single-player, Captions available, Steam Cloud]",7.99,False,664780,Choose Multiple LLC
18195,None,None,Altitude,Altitude,http://store.steampowered.com/app/620010/Altit...,2017-04-14,"[Movie, Detective, Action, Assassin]",http://steamcommunity.com/app/620010/reviews/?...,[Captions available],4.99,False,620010,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32071,None,None,Tank of War-VR,None,http://store.steampowered.com/app/745900/Tank_...,None,"[Action, Massively Multiplayer, Strategy, VR, ...",http://steamcommunity.com/app/745900/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",19.99,False,745900,None
32074,None,None,Flappy Arms,None,http://store.steampowered.com/app/764110/Flapp...,None,"[Casual, Simulation, VR, Funny, Colorful, Come...",http://steamcommunity.com/app/764110/reviews/?...,"[Single-player, Steam Leaderboards, HTC Vive, ...",1.99,False,764110,None
32075,None,None,SpaceWalker,None,http://store.steampowered.com/app/705860/Space...,None,"[Early Access, Casual]",http://steamcommunity.com/app/705860/reviews/?...,"[Single-player, HTC Vive, Oculus Rift, Tracked...",Free,True,705860,None
32083,None,None,LIV Client,None,http://store.steampowered.com/app/755540/LIV_C...,None,"[Video Production, Utilities, Web Publishing]",http://steamcommunity.com/app/755540/reviews/?...,"[Steam Workshop, Steam Cloud, HTC Vive, Oculus...",None,False,755540,None


In [16]:
steam.drop_duplicates(subset = 'id', keep = 'first', inplace = True)

We transform the 'release_date' values from str type to a Date type

In [17]:
# Passing errors = 'coerce' will force an out-of-bounds date to NaT, in addition to forcing 
# non-dates (or non-parseable dates) to NaT.
steam['release_date'] = pd.to_datetime(steam['release_date'], format = '%Y-%m-%d', errors = 'coerce')

In [18]:
steam['price']

0                4.99
1        Free To Play
2        Free to Play
3                0.99
4                2.99
             ...     
32128            1.99
32129            4.99
32130            1.99
32131            4.99
32132            4.99
Name: price, Length: 32132, dtype: object

We see that the price column has mixed type values, we normalise these values changing them to numbers

In [19]:
# We do a function just in case we have multiple columns/DataSets to change.
def toZero(var):
    """
    Take parameter and change it to zero, regardless of its type.

    parameters
    ----------
    n : str
        Non numeric value.

    examples
    --------
    >>> toZero('Hello')
    0
    >>> toZero(5)
    5.0
    """
    
    try:
        return float(var)
    except:
        return 0


In [20]:
steam['price'] = steam['price'].apply(toZero)

In [21]:
steam.head(1)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro


In [22]:
# Get rid of the unnecesary columns. This columns will not be used in our API.
steamColumns = ['publisher', 'app_name', 'url', 'reviews_url', 'early_access', 'specs', 'url', 'reviews_url']
steam.drop(steamColumns, axis = 1, inplace = True)

steam.head()

,genres,title,release_date,tags,price,id,developer
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140,Kotoshiro
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",0.00,643980,Secret Level SRL
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",0.00,670290,Poolians.com
3,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",0.99,767400,彼岸领域
4,None,None,NaT,"[Action, Indie, Casual, Sports]",2.99,773570,None


We procede to unnest the remaining list-columns into several columns.

In [23]:
# Check the key names.
reviews['reviews'][0]

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

In [24]:
reviewDF ={'user_id': [], 'funny': [],'posted': [], 'last_edited': [], 'item_id': [], 'helpful':[], 'recommend': [], 'review': [] }

for index, review in enumerate(reviews['reviews']):
    for r in list(review):
        for key, value in r.items():
            reviewDF[key].append(value)
        reviewDF['user_id'].append(reviews.iloc[index]['user_id'])

reviewDF = pd.DataFrame(reviewDF)
reviewDF.head()

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


Transform the text in the `posted` column into a Date value.

In [25]:
from datetime import datetime
reviewDF['posted'] = reviewDF['posted'].str.replace('Posted ', "")
reviewDF['posted'] = pd.to_datetime(reviewDF['posted'], format = '%B%d%Y.', errors = 'ignore')

In [26]:
reviewDF.drop(columns= ['funny', 'last_edited', 'helpful'], inplace = True)
reviewDF

,user_id,posted,item_id,recommend,review
0,76561197970982479,"November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,"April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,"September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...
57563,76561198312638244,July 10.,70,True,a must have classic from steam definitely wort...
57564,76561198312638244,July 8.,362890,True,this game is a perfect remake of the original ...
57565,LydiaMorley,July 3.,273110,True,had so much fun plaing this and collecting res...
57566,LydiaMorley,July 20.,730,True,:D


Unnest the items column in the items DataFrame

In [27]:
itemsDF = {'user_id': [], 'item_id': [], 'item_name': [], 'playtime_forever': [], 'playtime_2weeks': []}

for index, item_list in enumerate(items['items']):
    for item in item_list:
        itemsDF['user_id'].append(items.iloc[index]['user_id'])
        for key in itemsDF.keys():
            if key != 'user_id':
                itemsDF[key].append(item.get(key, None))  # Use get() to handle missing keys

itemsDF = pd.DataFrame(itemsDF)
itemsDF.head()


,user_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,6,0
1,76561197970982479,20,Team Fortress Classic,0,0
2,76561197970982479,30,Day of Defeat,7,0
3,76561197970982479,40,Deathmatch Classic,0,0
4,76561197970982479,50,Half-Life: Opposing Force,0,0


Apply sentiment analysis to the review column.

In [28]:
from textblob import TextBlob

def sentiment_analysis(column):
    '''
    Read text from a DataFrame, create a numeric column corresponding to previous one.
    This new column will have one for a neutral review, zero for bad review and two for a good review.

    Parameters
    ----------
    column : pandas.core.series.Series
        column name in which the analysis will be applied

    Returns
    -------
    DataFrame
        A DataFrame with a numeric column corresponding to review

    Examples
    --------
    >>> sentiment_analysis(positiveText)
    2
    >>> sentiment_analysis(negativeText)
    0
    >>> sentiment_analysis(neutralText)
    1
    '''
    analysis = TextBlob(column)
    if analysis.sentiment.polarity < 0:
        return 0
    elif analysis.sentiment.polarity == 0:
        return 1
    else:
        return 2

Apply the function to the DataFrame

In [29]:
reviewDF['sentiment_analysis'] = reviewDF['review'].apply(sentiment_analysis)
reviewDF['sentiment_analysis'].fillna(1, inplace = True)

Drop the review column, which will no longer be needed.

In [30]:
reviewDF.drop('review', axis = 1, inplace = True)

In [31]:
reviewDF.head(2)

,user_id,posted,item_id,recommend,sentiment_analysis
0,76561197970982479,"November 5, 2011.",1250,True,2
1,76561197970982479,"July 15, 2011.",22200,True,2


Save the DataFrames to .parquet extension files to save memory.

In [32]:
import parquet
import pyarrow
import fastparquet
reviewDF.to_parquet('./data/review.parquet', index = False)
itemsDF.to_parquet('./data/items.parquet', index = False)

Trying to save the steam DataFrame gives us an error, so we save it to csv format.

In [ ]:
steam.to_csv('./data/steam.csv', index = False, errors = 'replace')